In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data['Mood'] = ''

In [4]:
for i in range(len(data)):
    if data.at[i, 'label'] <= 0.25:
        data.at[i, 'Mood'] = 0
    if data.at[i, 'label'] > 0.25 and data.at[i, 'label'] <= 0.5:
        data.at[i, 'Mood'] = 1
    if data.at[i, 'label'] > 0.5 and data.at[i, 'label'] <= 0.75:
        data.at[i, 'Mood'] = 2
    if data.at[i, 'label'] > 0.75:
        data.at[i, 'Mood'] = 3

data['Mood'] = data['Mood'].astype(int)

In [5]:
data = data.drop(['label'], axis=1)

In [6]:
data.head()

,seq,Mood
0,"No, no\r\nI ain't ever trapped out the bando\r...",2
1,"The drinks go down and smoke goes up, I feel m...",2
2,She don't live on planet Earth no more\r\nShe ...,0
3,"Trippin' off that Grigio, mobbin', lights low\...",2
4,"I see a midnight panther, so gallant and so br...",1


In [22]:
data.at[0, 'seq']

"No, no\r\nI ain't ever trapped out the bando\r\nBut oh Lord, don't get me wrong\r\nI know a couple niggas that do\r\nI'm from a place where everybody knows your name\r\nThey say I gotta watch my attitude\r\nWhen they see money, man they all start actin' strange\r\nSo fuck with the ones that fuck with you\r\nThey can never say I'm brand new\r\n\r\nIt's everyday, everyday\r\nEveryday, everyday, everyday\r\nEveryday, everyday\r\nEveryday, everyday\r\nI've been talkin' my shit, nigga that's regular\r\nI've been lovin' 'em thick, life is spectacular\r\nI spend like I'ma die rich, nigga I'm flexin', yeah\r\nEveryday, that's everyday\r\nThat's everyday\r\nThat's everyday\r\nThat's everyday, everyday\r\n\r\nI see all of these wanna-be hot R&B singers\r\nI swear you all sound the same\r\nThey start from the bottom, so far from the motto\r\nYou niggas'll never be Drake\r\nShout out to OVO\r\nMost of them prolly don't know me though\r\nI stay in the cut, I don't fuck with no\r\nBody but I D, tha

In [7]:
import re
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [8]:
x = []
sentences = list(data["seq"])
for sen in sentences:
    x.append(preprocess_text(sen))

In [9]:
labels = data['Mood']
y = labels.values

In [10]:
tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(x)

x = tokenizer.texts_to_sequences(x)
# x_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

# X_test = pad_sequences(x_test, padding='post', maxlen=maxlen)

In [11]:
x = pad_sequences(x, padding='post', maxlen=maxlen)

In [12]:
from sklearn.model_selection import train_test_split

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [14]:
from sklearn.tree import DecisionTreeClassifier

In [15]:
dtr = DecisionTreeClassifier()

In [16]:
dtr.fit(x_train, y_train)

DecisionTreeClassifier()

In [17]:
from sklearn.metrics import classification_report

In [18]:
print(classification_report(y_test, dtr.predict(x_test)))

              precision    recall  f1-score   support

           0       0.32      0.34      0.33      9990
           1       0.34      0.34      0.34     14776
           2       0.33      0.32      0.33     13897
           3       0.26      0.25      0.26      8843

    accuracy                           0.32     47506
   macro avg       0.31      0.32      0.31     47506
weighted avg       0.32      0.32      0.32     47506



In [19]:
import pickle as pkl

In [21]:
with open("model.pkl", "wb") as file:
    pkl.dump(dtr, file)